# Feb 2, 2025: high resolution Allen parcellation CCFv3

In [1]:
import csv 
import os
import numpy as np
import pandas as pd
import ants
import seaborn as sns
import dill as pickle 

from allensdk.core.mouse_connectivity_cache import (
    MouseConnectivityCache,
    MouseConnectivityApi
)
from allensdk.api.queries.ontologies_api import OntologiesApi

import nrrd
import nibabel as nib

from brainrender import Atlas

In [2]:
# download CCFv3 template and annotations, and compare the orientation/affine metrics of annotation/template with that in brainrender

In [3]:
# ccfv3
atlas_path = f'{os.environ["HOME"]}/new_mouse_dataset/allen_atlas_ccfv3'
mcc_path = f'{atlas_path}/MouseConnectivity'
mcc = MouseConnectivityCache(
    resolution=25, # in micro meters (um)
    ccf_version=MouseConnectivityApi().CCF_2017,
    manifest_file=f'{mcc_path}/manifest.json',
)
AVGT, metaAVGT = mcc.get_template_volume()
ANO, metaANO = mcc.get_annotation_volume()
AVGT = AVGT.astype(np.float32)
ANO = ANO.astype(np.uint32)
print(AVGT.shape, ANO.shape)

STree = mcc.get_structure_tree()
STree_df = pd.DataFrame(STree.nodes()) 

(528, 320, 456) (528, 320, 456)


In [4]:
temp_file = f'{mcc_path}/average_template_25.nrrd'
data, header = nrrd.read(temp_file)

# affine information
spacing = header.get("space directions", np.eye(3))
origin = header.get("space origin", np.zeros(3))

# construct affine matrix for nifti
affine = np.eye(4)
affine[:3, :3] = spacing / 1000
affine[:3, 3] = origin

# convert to nifti
nifti_img = nib.Nifti1Image(data, affine)
nifti_file = f'{mcc_path}/average_template_25.nii.gz'
nib.save(nifti_img, nifti_file)

In [5]:
affine


array([[0.025, 0.   , 0.   , 0.   ],
       [0.   , 0.025, 0.   , 0.   ],
       [0.   , 0.   , 0.025, 0.   ],
       [0.   , 0.   , 0.   , 1.   ]])

In [6]:
temp_img = ants.image_read(nifti_file)
temp_img


ANTsImage (LPI)
         Pixel Type : float (float32)
         Components : 1
         Dimensions : (528, 320, 456)
         Spacing    : (0.025, 0.025, 0.025)
         Origin     : (0.0, 0.0, 0.0)
         Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

In [7]:
atlas = Atlas()
ref_vol = atlas.reference
ref_img = ants.from_numpy(
    data=ref_vol,
    origin=[0]*3,
    spacing=[25/1000]*3,
    direction=np.array([[1., 0., 0.],
                        [0., 1., 0.],
                        [0., 0., 1.]]),
)
ref_img.to_file(f'{mcc_path}/brainrender_ref_25.nii.gz')
ref_img


ANTsImage (RAI)
         Pixel Type : unsigned int (uint32)
         Components : 1
         Dimensions : (528, 320, 456)
         Spacing    : (0.025, 0.025, 0.025)
         Origin     : (0.0, 0.0, 0.0)
         Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]

In [8]:
atlas



╭──────────────────────────────────────────────────────────────────╮
│                         Allen mouse 25um                         │
│                                                                  │
│         name:  allen_mouse (v1.2)                                │
│      species:  Mus musculus                                      │
│     citation:  Wang et al 2020                                   │
│                https://doi.org/10.1016/j.cell.2020.04.007        │
│         link:  http://www.brain-map.org                          │
│                                                                  │
│  orientation:  asr                                               │
│    symmetric:  True                                              │
│   resolution:  [25.0, 25.0, 25.0]                                │
│        shape:  [528, 320, 456]                                   │
╰──────────────────────────────────────────────────────────────────╯